# Step 1: 

First we import some libraries

In [ ]:
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import scipy 
import numpy as np

# Step 2: 

loading the data 

In [ ]:
stitanic_test_file_path = 'test.csv'
stitanic_train_file_path = 'train.csv'

stitanic_data = pd.read_csv(stitanic_train_file_path)
df_test = pd.read_csv(stitanic_test_file_path)

# Step 3:

***Cleaning and exploring the data***

Note:
- When we copy one columns in Pandas, we also copy the missing value(NaN)
- So we need to fill the missing value before copy a collumn

***For Homeplanet***

- First, we will clean the data in Homplanet collumns.The function ***stitanic_data["HomePlanet"].value_counts()***  returns object containing counts of unique values 

HomePlanet    
Earth     4602  
Europa    2131
Mars      1759      
Name: count, dtype: int64
  
  
- After that, we will fill the collumn HomePlanet with the number occurs most often  
- Then, by using the function .get_dummies() , we will create new categorical columns : Earth, Europa and Mars  
For example :
  
   Homeplanet                                      
1 &emsp;    Earth                                        
2 &emsp;    Mars                 
3  &emsp;   Europa                                       
4   &emsp;  Mars                                         
will become 

&emsp;&nbsp; Earth &emsp; Mars &emsp; Europa    
1  &nbsp;1.00 &emsp;&nbsp; 0.00 &emsp;&nbsp; 0.00   
2  &nbsp;0.00 &emsp;&nbsp; 1.00 &emsp;&nbsp; 0.00   
3  &nbsp;0.00 &emsp;&nbsp; 0.00 &emsp;&nbsp; 1.00   
4  &nbsp;0.00 &emsp;&nbsp; 1.00 &emsp;&nbsp; 0.00   
- Finally we will drop the unecessary column : HomePlanet

In [ ]:
stitanic_data['HomePlanet'] = stitanic_data['HomePlanet'].fillna("Earth")
stitanic_homeplanet = pd.get_dummies(stitanic_data['HomePlanet'] , dtype = float)
stitanic_data = pd.concat([stitanic_data , stitanic_homeplanet ] ,axis = 1)
del stitanic_data["HomePlanet"]

We will do the same with the test data

In [ ]:
df_test['HomePlanet'] = df_test['HomePlanet'].fillna("Earth")
df_homeplanet = pd.get_dummies(df_test['HomePlanet'] , dtype = float)
df_test = pd.concat([df_test , df_homeplanet] ,axis = 1) 
del df_test['HomePlanet'] 

***For CryoSleep***  
We do the same thing with CryoSleep ***stitanic_data['CryoSleep'].value_counts()***  
Result: number of zero >number of 1 so we replace NaN = 0 

In [ ]:
stitanic_data['CryoSleep'] = stitanic_data['CryoSleep'].fillna(0)
stitanic_data['CryoSleep'] = stitanic_data['CryoSleep'].replace([False,True],[ 0 , 1 ])

df_test['CryoSleep'] = df_test['CryoSleep'].fillna(0)
df_test['CryoSleep'] = df_test['CryoSleep'].replace([False,True],[ 0 , 1 ])

***For Cabin***  
We have to split the type of Cabin from Deck/Num/side to column Deck , Num , side seperately.   
Because Cabin data are string data type so we can use string slicing 

In [ ]:
stitanic_data['C_Deck'] = stitanic_data['Cabin'].str[0] 
stitanic_data['C_Num'] = stitanic_data['Cabin'].str[2 : -2]
stitanic_data['C_Side'] = stitanic_data['Cabin'].str[-1]

# I don't know how to do yet.  
***For C_Num we have to change the dtype from object to categorical to use in the XGB model.***  
The two column C_Deck and C_side we do the same thing as ***Homeplanet*** column. 
We have to fill NaN in  C_Num, C_Deck, C_Side with mode 

In [ ]:
stitanic_data['C_Deck'] = stitanic_data['C_Deck'].fillna("F") 
stitanic_data['C_Num'] = stitanic_data['C_Num'].fillna(stitanic_data['C_Num'].mode()[0]) # fill with median or số xuất hiện nhiều nhất
stitanic_data['C_Side'] = stitanic_data['C_Side'].fillna("S")

stitanic_deck = pd.get_dummies(stitanic_data['C_Deck'], prefix = "C_Deck" , dtype = float)
stitanic_data = pd.concat([stitanic_data , stitanic_deck ] ,axis = 1)

stitanic_side = pd.get_dummies(stitanic_data['C_Side'], prefix = "C_Side" , dtype = float)
stitanic_data = pd.concat([stitanic_data , stitanic_side] ,axis = 1)

Finally is dropping column

In [ ]:
del stitanic_data['Cabin']
del stitanic_data['C_Deck']
del stitanic_data['C_Side']

In [ ]:
#test
#split -> deck , num , side 
df_test['C_Deck'] = df_test['Cabin'].str[0] 
df_test['C_Num'] = df_test['Cabin'].str[2 : -2]
df_test['C_Side'] = df_test['Cabin'].str[-1]

#đổi C_num từ dtype object sang float 
#fill
# F G  E  B  C  D  A  T
#print(df_test['C_Deck'].value_counts())
#fill

df_test['C_Deck'] = df_test['C_Deck'].fillna("F")
df_test['C_Num'] = df_test['C_Num'].fillna(df_test['C_Num'].mode()[0]) # fill with median or mode vì cnum ko phải là int
df_test['C_Side'] = df_test['C_Side'].fillna("S") # fill S hay random vì S 2093 P 2084

#
df_deck = pd.get_dummies(df_test['C_Deck'], prefix = "C_Deck", dtype = float)
df_test = pd.concat([df_test , df_deck],axis = 1) 
 
df_side = pd.get_dummies(df_test['C_Side'], prefix = "C_Side", dtype = float) # có nên để trong 1 cột ko ?
df_test = pd.concat([df_test , df_side],axis = 1) 

#drop col
del df_test['Cabin']
del df_test['C_Deck']
del df_test['C_Side']

***For Destination***
It is similar to HomePlanet

In [ ]:
stitanic_data['Destination'] = stitanic_data['Destination'].fillna('TRAPPIST-1e')
stitanic_destination = pd.get_dummies(stitanic_data['Destination'], dtype = float)
stitanic_data = pd.concat([stitanic_data ,stitanic_destination ],axis = 1) 
del stitanic_data['Destination']

#test
df_test['Destination'] = df_test['Destination'].fillna('TRAPPIST-1e')
df_destination = pd.get_dummies(df_test['Destination'], dtype = float)
df_test = pd.concat([df_test , df_destination],axis = 1) 
del df_test['Destination']

***For Ag, RoomService, FoodCourt, ShoppingMall, Spa, and VRDeck***  
We just simply replace the NaN value by the columns median

In [ ]:
stitanic_data['Age'] = stitanic_data['Age'].fillna(stitanic_data['Age'].median())
df_test['Age'] = df_test['Age'].fillna(df_test['Age'].median())

#Vip
#class bool
#stitanic_data['VIP'].value_counts()
#print( stitanic_data['VIP'].value_counts())  false > true 
stitanic_data['VIP'] = stitanic_data['VIP'].fillna(0)
stitanic_data['VIP'] = stitanic_data['VIP'].replace([False,True],[ 0 , 1 ])
df_test['VIP'] = df_test['VIP'].fillna(0)
df_test['VIP'] = df_test['VIP'].replace([False,True],[ 0 , 1 ])


#RoomService 
#class int
stitanic_data['RoomService'] = stitanic_data['RoomService'].fillna(stitanic_data['RoomService'].median())
df_test['RoomService'] = df_test['RoomService'].fillna(df_test['RoomService'].median())

#FoodCourt
#class int
stitanic_data['FoodCourt'] = stitanic_data['FoodCourt'].fillna(stitanic_data['FoodCourt'].median())
df_test['FoodCourt'] = df_test['FoodCourt'].fillna(df_test['FoodCourt'].median())

#ShoppingMall
#class int
stitanic_data['ShoppingMall'] = stitanic_data['ShoppingMall'].fillna(stitanic_data['ShoppingMall'].median())
df_test['ShoppingMall'] = df_test['ShoppingMall'].fillna (df_test['ShoppingMall'].median())

#Spa
#class int
stitanic_data['Spa'] = stitanic_data['Spa'].fillna(stitanic_data['Spa'].median())
df_test['Spa'] = df_test['Spa'].fillna( df_test['Spa'].median() )

#VRDeck

stitanic_data['VRDeck'] = stitanic_data['VRDeck'].fillna(stitanic_data['VRDeck'].median())
df_test['VRDeck'] = df_test['VRDeck'].fillna( df_test['VRDeck'].median() )

***For Name*** 

In [ ]:
del stitanic_data["Name"]
del df_test['Name']

# Step 4: Plotting variables 

***1. correlation analysis***

In [ ]:
stitanic_cor = stitanic_data.corr(method='pearson')
#sns.barplot(x=stitanic_cor['Transported'] , y= stitanic_cor.columns)

#heatmap but it cannot show columns so I use barchart
stitanc_cor_trans = stitanic_cor['Transported'].values.reshape(27,1)
plt.figure(figsize=(len (stitanic_data.columns) , len (stitanic_data.columns) ))
plt.title("correlation analysis")
sns.heatmap(data = stitanc_cor_trans , annot=True)

***2. FoodCourt***

In [ ]:
plt.figure(figsize=(14,6))
sns.lineplot(data=stitanic_data['FoodCourt'])

***3. Age***

In [ ]:
sns.barplot(x= len(stitanic_data['Age'], y=stitanic_data['Age'])

# Step 5: Building Model

First, We will use the ***accuracy_score*** to validate the model and splitting the data 

In [15]:
from sklearn.metrics import accuracy_score

#split data into train and test (8:2)

from sklearn.model_selection import train_test_split

stitanic_base =  [ item for item in stitanic_data.columns if item != 'Transported' ]
X = stitanic_data[stitanic_base] 
y = stitanic_data['Transported']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2 , random_state = 42)

Secondly, I will try 3 model ,decision tree , logistic regerssion, and xgboost, to see which one is the best 

 - The first one is logistic regression.

In [ ]:
from sklearn.linear_model import LogisticRegression

model_logistic = LogisticRegression()
model_logistic.fit(X_train, y_train)
predict_log_train = model_logistic.predict(X_train)
predict_log_test = model_logistic.predict(X_test) 
predict_log_ans = model_logistic.predict(df_test)

We need to validate the model by creating a small test set from the training set.  
Then, we print the result out to select the best model  
We will choose the model with highest performance in the test_log_pt  

In [ ]:
train_log_pt = accuracy_score (predict_log_train , y_train )
test_log_pt = accuracy_score (predict_log_test , y_test )
print ( train_log_pt, test_log_pt , sep = '\n')

 - The second one is decision tree   
  &emsp; We need to choose the max_depth of a decision tree  
  &emsp; The max_depth = 5 return the best result   

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_tree = DecisionTreeClassifier( max_depth = 5, random_state = 1)
model_tree.fit(X_train , y_train)
predict_tree_train = model_tree.predict(X_train)
predict_tree_test= model_tree.predict(X_test)
predict_tree_ans = model_tree.predict(df_test)
train_tree_pt = accuracy_score(predict_tree_train , y_train)
test_tree_pt = accuracy_score(predict_tree_test , y_test)
print ( train_tree_pt , test_tree_pt , sep = '\n')

 - The third one is xgboost  
    &emsp; We need to use ._get_numeric_data() to fit the xgboost, this will drop the C_Num and PassengerId columns 

In [ ]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier()
#xgb ko nhận được dtype :object nên phải dùng ._get_numeric_data()
#nó sẽ drop C_num và passengerId
model_xgb.fit(X_train._get_numeric_data() , y_train)
predict_xgb_train = model_xgb.predict(X_train._get_numeric_data())
predict_xgb_test= model_xgb.predict(X_test._get_numeric_data())
predict_xgb_ans = model_xgb.predict(df_test._get_numeric_data())
train_xgb_pt = accuracy_score(predict_xgb_train , y_train)
test_xgb_pt = accuracy_score(predict_xgb_test , y_test)
#print ( train_xgb_pt , test_xgb_pt , sep = '\n')

# Step 6 : Submission

 &emsp; The second line is for xgb because it automatically converted the Transported column into 0 and 1.

In [ ]:
df_test["Transported"] = predict_log_ans
#df_test["Transported"] = df_test["Transported"].replace([0,1],[ False, True ])
csv = df_test[['PassengerId',"Transported"]]

csv.to_csv("sub_stitanic_logistic_2.csv", index = False)
'''

In [88]:
#Some useful command

#predict_ans = model_logistic.predict(df_test)
#print(predict_ans)

#print(stitanic_data)
#missing_cnt = df_test.isnull().sum()

#print(stitanic_data['Transported'].head())

#print(missing_cnt)

HomePlanet
Earth     4602
Europa    2131
Mars      1759
Name: count, dtype: int64


'\n#submission xgb\ndf_test["Transported"] = predict_ans\n#df_test["Transported"] = df_test["Transported"].replace([0,1],[ False, True ])\ncsv = df_test[[\'PassengerId\',"Transported"]]\n\ncsv.to_csv("sub_stitanic_logistic_2.csv", index = False)\n'